# 5-2 | 교차 검증과 그리드 서치 |

In [667]:
import pandas as pd
wine = pd.read_csv("https://bit.ly/wine-date")

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [668]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [669]:
sub_data, val_data, sub_target, val_target = train_test_split(train_data, train_target, test_size=0.2, random_state=42)

In [670]:
print(sub_data.shape, val_data.shape, test_data.shape)

(4157, 3) (1040, 3) (1300, 3)


In [671]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_data, sub_target)
print(dt.score(sub_data, sub_target))
print(dt.score(val_data, val_target)) #overfittiing

0.9971133028626413
0.864423076923077


In [672]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_data, train_target)
print(scores)

{'fit_time': array([0.00498581, 0.00498605, 0.0050199 , 0.00598836, 0.00501966]), 'score_time': array([0.        , 0.00097036, 0.00099087, 0.        , 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [673]:
import numpy as np
print(np.mean(scores['test_score'])) #검증폴드의 평균 점수

0.855300214703487


In [674]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_data, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [675]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_data, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [676]:
# 그리디 서치(Grid Search)

from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

In [677]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)

In [678]:
gs.fit(train_data, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [679]:
dt = gs.best_estimator_
print(dt.score(train_data, train_target))
print(gs.best_params_)
print(gs.cv_results_['mean_test_score'])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [680]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


1. 탐색할 매개변수를 지정
2. 훈련세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음
3. 그리드 서치는 최상의 매개변수에서 전체 훈련세트를 사용해 최종 모델을 훈련

In [681]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001,0.0001), 'max_depth' : range(5, 20, 1), 'min_samples_split' : range(2, 100, 10)}

In [682]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_data, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [683]:
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([3, 5, 1, 5, 6, 2, 2, 6, 0, 0])

In [684]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([103, 100, 106, 121, 104,  85, 106,  90,  93,  92], dtype=int64))

In [685]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.60350118, 0.19209929, 0.66478917, 0.68595876, 0.31659316,
       0.87999239, 0.90453522, 0.78238063, 0.04963808, 0.61855275])

In [686]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [687]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_data, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E692724B50>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E692709850>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E692924D30>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E692876040>},
                   random_state=42)

In [688]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [689]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [690]:
dt = gs.best_estimator_
print(dt.score(test_data, test_target))

0.86
